# Modelos

In [1]:
# Importando bibliotecas
from sklearn import datasets
from sklearn.model_selection import train_test_split
import pandas as pd

# Carregando o dataset Iris
iris = datasets.load_iris()

# Transformando em DataFrame para facilitar a visualização
df_iris = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df_iris['target'] = iris.target

In [3]:
df_iris

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [5]:
X = df_iris.drop(columns=["target"])
y = df_iris["target"]

- **X**: São as features (variáveis preditoras) do dataset Iris, ou seja, as medidas das sépalas e pétalas.- 
- **y**: São as classes (rótulos, variável resposta) do dataset Iris, ou seja, as espécies de flores.

In [8]:
X

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [9]:
y

0      0
1      0
2      0
3      0
4      0
      ..
145    2
146    2
147    2
148    2
149    2
Name: target, Length: 150, dtype: int64

## Data splitting

In [11]:
# Dividindo o dataset em treino (60%), calibração (20%) e teste (20%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_calib, X_test, y_calib, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

## Por que separamos em treino, calibração e teste?

A separação dos dados em **treino**, **calibração** e **teste** é uma prática essencial em Machine Learning, pois garante que o modelo seja bem treinado e avaliado de maneira correta e justa. Abaixo explicamos o propósito de cada conjunto de dados:

#### 1. Conjunto de Treinamento (Treino)
- **Função**: O conjunto de treino é utilizado para **ajustar os parâmetros do modelo** (por exemplo, os coeficientes em uma regressão linear ou os nós de uma árvore de decisão).
- **Motivo**: Durante o treinamento, o modelo "aprende" a partir dos dados de entrada, identificando padrões que são utilizados para fazer previsões. Se o modelo só treinasse e fosse avaliado nos mesmos dados, poderíamos ter um resultado enviesado, conhecido como **overfitting**.

#### 2. Conjunto de Calibração (Validação ou Calibração)
- **Função**: O conjunto de calibração é utilizado para **ajustar hiperparâmetros** do modelo, como a profundidade máxima de uma árvore de decisão, o número de vizinhos no KNN, ou o fator de regularização em uma regressão logística.
- **Motivo**: Calibrar o modelo ajuda a evitar **overfitting** ou **underfitting**. Isso garante que o modelo não apenas tenha um bom desempenho nos dados de treino, mas também seja capaz de generalizar para novos dados. Sem um conjunto de calibração, poderíamos escolher hiperparâmetros que maximizam o desempenho no treino, mas que falham em dados novos.

#### 3. Conjunto de Teste
- **Função**: O conjunto de teste é utilizado para **avaliar a performance final** do modelo após o treino e a calibração, sem afetar o modelo.
- **Motivo**: O conjunto de teste representa dados completamente novos para o modelo, simulando como ele se comportaria em situações reais, onde ele não tem acesso prévio às informações. Ao avaliar o modelo com o conjunto de teste, podemos verificar se ele generaliza bem para dados desconhecidos, sem o risco de overfitting.
